# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms, \
    compute_affine_transformation, compute_affine_transformation_centroid
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


xgboost version=2.1.4


In [3]:
def absolute_sum(l):
    la = np.array(l)
    nabs = (np.array(la*la))
    return np.sum(la, axis=0)

def sum_square_com(com):
    ss = np.sqrt(sum([s*s for s in com]))
    return ss

In [15]:
atlas_all = list_coms('AtlasV7')
allen_all = list_coms('Allen')
common_keys = list(atlas_all.keys() & allen_all.keys())
atlas_common = np.array([atlas_all[s] for s in common_keys])
allen_common = np.array([allen_all[s] for s in common_keys])
print(len(common_keys))

37


In [5]:
base_keys = ['LRt_L', 'LRt_R', 'SC', 'IC']
test_keys = ['PBG_L', 'PBG_R', 'SC', 'IC', '4N_L', '4N_R', 'LRt_L','LRt_R', 'SNC_L', 'SNC_R', ]
bad_keys = ['RtTg','AP']
good_keys = set(common_keys) - set(bad_keys)
print(len(base_keys))
print(len(test_keys))
print(len(bad_keys))
print(len(good_keys))
print(len(common_keys))

4
10
2
35
37


In [16]:
atlas_src = np.array([atlas_all[s] for s in common_keys])
allen_src = np.array([allen_all[s] for s in common_keys])
transformation_matrix = compute_affine_transformation(atlas_src, allen_src)

In [17]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = np.array(allen_all[structure]) 
    transformed = apply_affine_transform(atlas0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, allen0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(atlas0), np.round(allen0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# test keys RMS 25.61709479307028
# base keys RMS 25.602246865972344

RMS 259.9671780610904


In [21]:
# [10266.  5055.  6843.]
scales = np.array([0.452*32, 0.452*32, 20])
sc_atlas = np.array([709.77,349.50,342.13])
print(f'new sc {sc_atlas * scales}')

structure = 'SC'
print(f'Atlas {structure} {np.round(np.array(atlas_all[structure]))}')
print(f'Allen {structure} {np.round(np.array(allen_all[structure]))}')
print(f'Atlas transformed {structure} {np.round(np.array(transformed_dict[structure]))}')

new sc [10266.11328  5055.168    6842.6    ]
Atlas SC [7776. 2654. 5730.]
Allen SC [9140. 2388. 5692.]
Atlas transformed SC [9161. 2385. 5718.]


In [9]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['structure'], inplace=True)
HTML(df.to_html(index=False))

structure,atlas0,allen0,transformed,difference,sumsquares
3N_L,"[7756.0, 3915.0, 5574.0]","[9102.0, 3794.0, 5521.0]","[9206.0, 3890.0, 5553.0]","[104.0, 96.0, 32.0]",144.76
3N_R,"[7756.0, 3915.0, 5826.0]","[9103.0, 3794.0, 5864.0]","[9207.0, 3892.0, 5837.0]","[104.0, 98.0, -28.0]",145.39
4N_L,"[8092.0, 3967.0, 5452.0]","[9589.0, 3778.0, 5432.0]","[9525.0, 3951.0, 5410.0]","[-64.0, 173.0, -22.0]",185.83
4N_R,"[8092.0, 3967.0, 5948.0]","[9587.0, 3775.0, 5953.0]","[9527.0, 3955.0, 5970.0]","[-60.0, 180.0, 17.0]",190.28
5N_L,"[8668.0, 5021.0, 4329.0]","[10193.0, 5287.0, 4092.0]","[10116.0, 5201.0, 4142.0]","[-77.0, -86.0, 50.0]",125.67
5N_R,"[8668.0, 5021.0, 7071.0]","[10193.0, 5286.0, 7293.0]","[10127.0, 5220.0, 7240.0]","[-66.0, -65.0, -53.0]",107.00
6N_L,"[9262.0, 5124.0, 5338.0]","[10771.0, 5215.0, 5291.0]","[10686.0, 5330.0, 5274.0]","[-86.0, 115.0, -17.0]",144.37
6N_R,"[9262.0, 5124.0, 6062.0]","[10771.0, 5215.0, 6094.0]","[10688.0, 5335.0, 6092.0]","[-83.0, 120.0, -2.0]",146.13
7N_L,"[9395.0, 6157.0, 4558.0]","[10853.0, 6775.0, 4340.0]","[10859.0, 6558.0, 4399.0]","[7.0, -218.0, 59.0]",225.75
7N_R,"[9395.0, 6157.0, 6842.0]","[10853.0, 6775.0, 7045.0]","[10869.0, 6574.0, 6980.0]","[16.0, -201.0, -65.0]",212.12


In [10]:
df['sumsquares'].sum()/len(df.index)

259.96675675675675